In [1]:
# Import modules

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
# Import and sort train & test sets

train = pd.read_csv('../data/train/train_data')
train = train.sort_values('img_id', ascending=True)

test = pd.read_csv('../data/test/test_data')
test = test.sort_values('img_id', ascending=True)

In [3]:
# Import train & test images

train_image = []
test_image = []

for i in tqdm(range(train.shape[0])):
    try:
        img = image.load_img('../data/train/train_img/' + train['img_id'][i].strip("0"), target_size=(28,28,1), grayscale=True)
        img = image.img_to_array(img)
        img = img/255
        train_image.append(img)
    except:
        img = image.load_img('../data/train/train_img/0.jpg', target_size=(28,28,1), grayscale=True)
        img = image.img_to_array(img)
        img = img/255
        train_image.append(img)
        
for i in tqdm(range(test.shape[0])): 
    try:
        img = image.load_img('../data/test/test_img/' + test['img_id'][i].strip("0"), target_size=(28,28,1), grayscale=True)
        img = image.img_to_array(img)
        img = img/255
        test_image.append(img)
    except:
        img = image.load_img('../data/test/test_img/0.jpg', target_size=(28,28,1), grayscale=True)
        img = image.img_to_array(img)
        img = img/255
        test_image.append(img)

X_train = np.array(train_image)
X_test = np.array(test_image)

print(X_train)

  0%|          | 0/14034 [00:00<?, ?it/s]/Users/vtamprateep/opt/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 3000/3000 [00:05<00:00, 578.10it/s]


[[[[0.46666667]
   [0.7137255 ]
   [0.5647059 ]
   ...
   [0.3764706 ]
   [0.3647059 ]
   [0.36078432]]

  [[0.1254902 ]
   [0.45882353]
   [0.3019608 ]
   ...
   [0.4       ]
   [0.39215687]
   [0.3882353 ]]

  [[0.01960784]
   [0.08235294]
   [0.7254902 ]
   ...
   [0.43137255]
   [0.42352942]
   [0.40784314]]

  ...

  [[0.16078432]
   [0.12156863]
   [0.2627451 ]
   ...
   [0.20392157]
   [0.17254902]
   [0.17254902]]

  [[0.15686275]
   [0.11764706]
   [0.3137255 ]
   ...
   [0.15294118]
   [0.14509805]
   [0.10980392]]

  [[0.07450981]
   [0.23921569]
   [0.14901961]
   ...
   [0.12156863]
   [0.13333334]
   [0.13333334]]]


 [[[0.46666667]
   [0.00784314]
   [0.7137255 ]
   ...
   [0.4509804 ]
   [0.14117648]
   [0.3019608 ]]

  [[0.05490196]
   [0.30980393]
   [0.92156863]
   ...
   [0.2627451 ]
   [0.2784314 ]
   [0.21176471]]

  [[0.7019608 ]
   [0.7372549 ]
   [0.41960785]
   ...
   [0.7294118 ]
   [0.49411765]
   [0.32941177]]

  ...

  [[0.2627451 ]
   [0.40392157]
   [0.1

In [4]:
# OneHotEncode categories

y_train = train['class'].values
y_test = test['class'].values

l_encoder = LabelEncoder()
l_encoder.fit(y_train)
print(l_encoder.classes_)

y_train = l_encoder.transform(y_train)
y_test = l_encoder.transform(y_test)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(y_train)

['buildings' 'forest' 'glacier' 'mountain' 'sea' 'street']
[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 ...
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [5]:
# Create the model

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

checkpoint_path = "../model/checkpoint"
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

In [6]:
# Train model

model.fit(X_train, y_train, epochs = 15, validation_data=(X_test, y_test))

Epoch 1/15
439/439 [==============================] - 21s 47ms/step - loss: 1.7922 - accuracy: 0.1748 - val_loss: 1.7902 - val_accuracy: 0.1750
Epoch 2/15
439/439 [==============================] - 31s 70ms/step - loss: 1.7917 - accuracy: 0.1776 - val_loss: 1.7902 - val_accuracy: 0.1767
Epoch 3/15
439/439 [==============================] - 33s 75ms/step - loss: 1.7910 - accuracy: 0.1801 - val_loss: 1.7912 - val_accuracy: 0.1730
Epoch 4/15
439/439 [==============================] - 36s 82ms/step - loss: 1.7879 - accuracy: 0.1828 - val_loss: 1.7941 - val_accuracy: 0.1660
Epoch 5/15
439/439 [==============================] - 36s 82ms/step - loss: 1.7778 - accuracy: 0.1994 - val_loss: 1.7998 - val_accuracy: 0.1633
Epoch 6/15
439/439 [==============================] - 37s 85ms/step - loss: 1.7523 - accuracy: 0.2212 - val_loss: 1.8184 - val_accuracy: 0.1680
Epoch 7/15
439/439 [==============================] - 36s 83ms/step - loss: 1.7096 - accuracy: 0.2552 - val_loss: 1.8332 - val_accuracy:

In [14]:
# Try test image

test_image = image.load_img("../data/pred/pred/3.jpg", target_size=(28, 28, 1), grayscale=True)
test_image = image.img_to_array(test_image)
test_image = test_image/255
test_image = np.array([test_image])

prediction = np.argmax(model.predict(test_image), axis=1)

In [15]:
print(prediction)

[1]


In [16]:
# Save model

model.save_weights("../model/checkpoint/model_checkpoint")